In [1]:
from astropy.time import Time #to time converting
from astropy import units #time correction
from astropy.coordinates import SkyCoord #time correction
import math #to convertion calculus
import numpy as np
import os.path
from tabulate import tabulate
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import sncosmo

In [2]:
def radec_to_xyz(ra, dec):
    x = math.cos(np.deg2rad(dec)) * math.cos(np.deg2rad(ra))
    y = math.cos(np.deg2rad(dec)) * math.sin(np.deg2rad(ra))
    z = math.sin(np.deg2rad(dec))

    return np.array([x, y, z], dtype=np.float64)

#functions from sncosmo
def cmb_dz(ra, dec):
    """See http://arxiv.org/pdf/astro-ph/9609034
     CMBcoordsRA = 167.98750000 # J2000 Lineweaver
     CMBcoordsDEC = -7.22000000
    """

    # J2000 coords from NED
    CMB_DZ = 371000. / 299792458.
    CMB_RA = 168.01190437
    CMB_DEC = -6.98296811
    CMB_XYZ = radec_to_xyz(CMB_RA, CMB_DEC)

    coords_xyz = radec_to_xyz(ra, dec)
    
    dz = CMB_DZ * np.dot(CMB_XYZ, coords_xyz)

    return dz

def cmb_to_helio(z, ra, dec):
    """Convert from CMB-frame redshift to heliocentric redshift.
    
    Parameters
    ----------
    z : float
        CMB-frame redshift.
    ra, dec: float
        RA and Declination in degrees (J2000).
    """

    dz = -cmb_dz(ra, dec)
    one_plus_z_pec = math.sqrt((1. + dz) / (1. - dz))
    one_plus_z_helio = (1. + z) * one_plus_z_pec

    return one_plus_z_helio - 1.

In [3]:
kernel = 0.5 * (RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)))

In [4]:
#time, flux, flux error and bands
x = []
zb = []
zberr = []
zg = []
zgerr = []
zr = []
zrerr = []
zi = []
zierr = []

In [5]:
#reading the file
file = open("SN2007jgopt+nir_photo.dat")
lines = file.readlines()
file.close()

for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    #nome.append(co[])
    #band.append(co[1])
    x.append(co[0])
    
    zb.append(co[3])
    zberr.append(co[4])
    
    zg.append(co[7])
    zgerr.append(co[8])
    
    zr.append(co[9])
    zrerr.append(co[10])
    
    zi.append(co[11])
    zierr.append(co[12])
    
x = [float(i) for i in x]

zb = [float(i) for i in zb]
zberr = [float(i) for i in zberr]

zg = [float(i) for i in zg]
zgerr = [float(i) for i in zgerr]

zr = [float(i) for i in zr]
zrerr = [float(i) for i in zrerr]

zi = [float(i) for i in zi]
zierr = [float(i) for i in zierr]

In [6]:
zcmb = float(lines[2][9:17])

In [7]:
ra = lines[2][24:35]

In [8]:
dec = lines[2][43:55]

In [9]:
ra

' 03:29:50.8'

In [10]:
dec

' +00:03:24.6'

In [11]:
c = SkyCoord(str(ra), str(dec), unit=(units.hourangle, units.hourangle))

In [12]:
zhelio = cmb_to_helio(zcmb, c.ra.deg, c.dec.deg)

In [13]:
xb = []
zbmod = []
zberrmod = []

xg = []
zgmod = []
zgerrmod = []

xr = []
zrmod = []
zrerrmod = []

xi = []
zimod = []
zierrmod = []

In [14]:
for i in range(0, len(x)):
    
    if zb[i] != 99.900:
        
        xb.append([x[i]])
        zbmod.append(zb[i])
        zberrmod.append(zberr[i])
        
    if zg[i] != 99.900:
        
        xg.append([x[i]])
        zgmod.append(zg[i])
        zgerrmod.append(zgerr[i])

    if zr[i] != 99.900:
        
        xr.append([x[i]])
        zrmod.append(zr[i])
        zrerrmod.append(zrerr[i])
        
    if zi[i] != 99.900:
        
        xi.append([x[i]])
        zimod.append(zi[i])
        zierrmod.append(zierr[i])

In [15]:
csp = sncosmo.get_magsystem('csp')

In [16]:
for i in range(0,len(xb)):
    
    #flux = 10**((14.328 - zbmod[i])/2.5)
    flux = csp.band_mag_to_flux(zbmod[i], 'cspb')
    zbmod[i] = flux
    zberrmod[i] = flux*zberrmod[i]*np.log(10)/2.5

In [17]:
for i in range(0,len(xg)):
    
    #flux = 10**((15.111 - zgmod[i])/2.5)
    flux = csp.band_mag_to_flux(zgmod[i], 'cspg')
    zgmod[i] = flux
    zgerrmod[i] = flux*zgerrmod[i]*np.log(10)/2.5

In [18]:
for i in range(0,len(xr)):
    
    #flux = 10**((14.902 - zrmod[i])/2.5)
    flux = csp.band_mag_to_flux(zrmod[i], 'cspr')
    zrmod[i] = flux
    zrerrmod[i] = flux*zrerrmod[i]*np.log(10)/2.5

In [19]:
for i in range(0,len(xi)):
    
    #flux = 10**((14.545 - zimod[i])/2.5)
    flux = csp.band_mag_to_flux(zimod[i], 'cspi')
    zimod[i] = flux
    zierrmod[i] = flux*zierrmod[i]*np.log(10)/2.5

In [20]:
table = []

In [21]:
X_data = np.linspace(min(xb)[0], max(xb)[0], 610)
gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(xb, zbmod)
y_mean = gp.predict(X_data[:, np.newaxis])    


pos_max = y_mean.tolist().index(max(y_mean))
print(pos_max)
t0_max = X_data[pos_max]
    
for i in range(0,len(xb)):  
    
    if -5 < xb[i] - t0_max < 45:
    
        table.append((xb[i][0], "cspb", zbmod[i], zberrmod[i], 0, "csp", zhelio))

30


In [22]:
X_data = np.linspace(min(xg)[0], max(xg)[0], 610)
gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(xg, zgmod)
y_mean = gp.predict(X_data[:, np.newaxis])    

pos_max = y_mean.tolist().index(max(y_mean))
print(pos_max)
t0_max = X_data[pos_max]
    
for i in range(0,len(xg)):  
    
    if -5 < xg[i] - t0_max < 45:
    
        table.append((xg[i][0], "cspg", zgmod[i], zgerrmod[i], 0, "csp", zhelio))

32


In [23]:
X_data = np.linspace(min(xr)[0], max(xr)[0], 610)
gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(xr, zrmod)
y_mean = gp.predict(X_data[:, np.newaxis])    

pos_max = y_mean.tolist().index(max(y_mean))
print(pos_max)
t0_max = X_data[pos_max]
    
for i in range(0,len(xr)):  
    
    if -5 < xr[i] - t0_max < 45:
    
        table.append((xr[i][0], "cspr", zrmod[i], zrerrmod[i], 0, "csp", zhelio))

48


In [24]:
X_data = np.linspace(min(xi)[0], max(xi)[0], 610)
gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(xi, zimod)
y_mean = gp.predict(X_data[:, np.newaxis])    

pos_max = y_mean.tolist().index(max(y_mean))
print(pos_max)
t0_max = X_data[pos_max]
    
for i in range(0,len(xi)):  
    
    if -5 < xi[i] - t0_max < 45:
    
        table.append((xi[i][0], "cspi", zimod[i], zierrmod[i], 0, "csp", zhelio))

0


In [25]:
save_path = '/home/joao/Documentos/LC/'

In [26]:
name_of_file = "SN2007jg"

completeName = os.path.join(save_path, name_of_file+".txt")    

header = "#time    band  flux        fluxerr      zp     zpsys redshift"

f = open(completeName, 'w')               
f.write(header + "\n")
f.write(tabulate(table, tablefmt="plain"))
f.close()

In [27]:
zhelio

0.03712857164683747